In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

import sys
sys.path.append('Config')
from lablib import *

### Funzioni utili

In [ ]:
root_path = r'Dati/Parte 2/Caratterizzazione_impulso/'
skip = 18
errore_medio = 0.015 / np.sqrt(12)  

def read_data(filename, titles = ['Time', 'V']):
    df = read_corretto(root_path + filename, skip, titles)

    # Sostituire tutte le istanze di "," con "" nelle colonne: 'V', 'Time'
    df['V'] = df['V'].str.replace(",", "", case=False, regex=False)
    df['Time'] = df['Time'].str.replace(",", "", case=False, regex=False)

    # Converte le colonne in float
    df = df.astype(float)

    return df

def _crop_df(df, N, thr):
    df = crop_df(df, N, thr)
    df.index = np.arange(len(df))
    return df


def quick_plot(df):
    plt.plot(df['Time'], df['V'])
    plt.xlabel('Time')
    plt.ylabel('V')
    plt.title('Signal')
    plt.show()

def find_max(df):
    max_index = df['V'].idxmax()
    max_value = df['V'][max_index]

    return max_index, max_value

def exp_dec(x, a, b, c):
    return a * np.exp(-b * x) + c

def exp_cre(x, a, b, c):
    return a * (1 - np.exp(-b * x)) + c

def rise_fall(df):
    max_index, _ = find_max(df)

    rise_time = df['Time'][:max_index]
    fall_time = df['Time'][max_index:]
    rise_voltage = df['V'][:max_index]
    fall_voltage = df['V'][max_index:]

    rise = pd.DataFrame({'Time': rise_time, 'V': rise_voltage})
    fall = pd.DataFrame({'Time': fall_time, 'V': fall_voltage})
    return rise, fall



## BGO

### Na

In [ ]:
bgo_na = pd.read_csv(root_path + 'BGO_Na22.CSV')
bgo_na = _crop_df(bgo_na, 10, .015)

bgo_na_rise, bgo_na_fall = rise_fall(bgo_na)
bgo_na_ers1 = [errore_medio] * len(bgo_na_rise)
bgo_na_ers2 = [errore_medio] * len(bgo_na_fall)

plt.plot(bgo_na_rise['Time'], bgo_na_rise['V'], label='Rise')
plt.plot(bgo_na_fall['Time'], bgo_na_fall['V'], label='Fall')
plt.show()

In [ ]:
pars1, vals1, ers1, *other1 = LS_fit(bgo_na_rise['Time'], bgo_na_rise['V'], bgo_na_ers1, exp_cre, 
                                            #limits = [['a', (0, 1)], ['b', (10_0000, None)]],
                                            a=.0035, b=21e6, c=0) # era solo un problema di inizializzazione

In [ ]:
pars2, vals2, ers2, *other2 = LS_fit(bgo_na_fall['Time'], bgo_na_fall['V'], bgo_na_ers2, exp_dec, a=1, b=0.01, c=0)

In [ ]:
tau_bgo_na_rise = 1 / vals1[1] * 1e9
err_bgo_na_rise = 1 / vals1[1]**2 * ers1[1] * 1e9

tau_bgo_na_fall = 1 / vals2[1] * 1e9
err_bgo_na_fall = 1 / vals2[1]**2 * ers2[1] * 1e9

### Co

In [ ]:
bgo_co = pd.read_csv(root_path + 'BGO_Co60.CSV')
bgo_co = _crop_df(bgo_co, 10, .01)

bgo_co_rise, bgo_co_fall = rise_fall(bgo_co)
bgo_co_ers1 = [errore_medio] * len(bgo_co_rise)
bgo_co_ers2 = [errore_medio] * len(bgo_co_fall)

plt.plot(bgo_co_rise['Time'], bgo_co_rise['V'], label='Rise')
plt.plot(bgo_co_fall['Time'], bgo_co_fall['V'], label='Fall')
plt.show()

In [ ]:
pars3, vals3, ers3, *other3 = LS_fit(bgo_co_rise['Time'], bgo_co_rise['V'], bgo_co_ers1, exp_cre,
                                            a=.0035, b=21e6, c=0)
pars4, vals4, ers4, *other4 = LS_fit(bgo_co_fall['Time'], bgo_co_fall['V'], bgo_co_ers2, exp_dec,
                                            a=1, b=0.01, c=0)

In [ ]:
tau_bgo_co_rise = 1 / vals3[1] * 1e9
err_bgo_co_rise = 1 / vals3[1]**2 * ers3[1] * 1e9

tau_bgo_co_fall = 1 / vals4[1] * 1e9
err_bgo_co_fall = 1 / vals4[1]**2 * ers4[1] * 1e9

## CSI

### Na

In [ ]:
csi_na = pd.read_csv(root_path + 'CSI_Na22.CSV')
csi_na = _crop_df(csi_na, 10, .03)

csi_na_rise, csi_na_fall = rise_fall(csi_na)
csi_na_ers1 = [errore_medio] * len(csi_na_rise)
csi_na_ers2 = [errore_medio] * len(csi_na_fall)

plt.plot(csi_na_rise['Time'], csi_na_rise['V'], label='Rise')
plt.plot(csi_na_fall['Time'], csi_na_fall['V'], label='Fall')
plt.show()

In [ ]:
pars5, vals5, ers5, *other5 = LS_fit(csi_na_rise['Time'], csi_na_rise['V'], csi_na_ers1, exp_cre,
                                            a=.0035, b=21e6, c=0)
pars6, vals6, ers6, *other6 = LS_fit(csi_na_fall['Time'], csi_na_fall['V'], csi_na_ers2, exp_dec,
                                            a=1, b=0.01, c=0)

In [ ]:
tau_csi_na_rise = 1 / vals5[1] * 1e9
err_csi_na_rise = 1 / vals5[1]**2 * ers5[1] * 1e9

tau_csi_na_fall = 1 / vals6[1] * 1e9
err_csi_na_fall = 1 / vals6[1]**2 * ers6[1] * 1e9

### Co

In [ ]:
csi_co = pd.read_csv(root_path+'CSI_Co60.CSV')
csi_co = _crop_df(csi_co, 10, .03)

csi_co_rise, csi_co_fall = rise_fall(csi_co)
csi_co_ers1 = [errore_medio] * len(csi_co_rise)
csi_co_ers2 = [errore_medio] * len(csi_co_fall)

plt.plot(csi_co_rise['Time'], csi_co_rise['V'], label='Rise')
plt.plot(csi_co_fall['Time'], csi_co_fall['V'], label='Fall')
plt.show()

In [ ]:
pars7, vals7, ers7, *other7 = LS_fit(csi_co_rise['Time'], csi_co_rise['V'], csi_co_ers1, exp_cre,
                                            a=.0035, b=21e6, c=0)
pars8, vals8, ers8, *other8 = LS_fit(csi_co_fall['Time'], csi_co_fall['V'], csi_co_ers2, exp_dec,
                                            a=1, b=0.01, c=0)

In [ ]:
tau_csi_co_rise = 1 / vals7[1] * 1e9
err_csi_co_rise = 1 / vals7[1]**2 * ers7[1] * 1e9

tau_csi_co_fall = 1 / vals8[1] * 1e9
err_csi_co_fall = 1 / vals8[1]**2 * ers8[1] * 1e9

## Confronto

__Confronto tempi di discesa__

In [ ]:
# Discesa

print(f'Fall BGO NA: {tau_bgo_na_fall:.2f} ± {err_bgo_na_fall:.2f} ns')
print(f'Fall BGO CO: {tau_bgo_co_fall:.2f} ± {err_bgo_co_fall:.2f} ns')
print('')
print(f'Fall CSI NA: {tau_csi_na_fall:.2f} ± {err_csi_na_fall:.2f} ns')
print(f'Fall CSI CO: {tau_csi_co_fall:.2f} ± {err_csi_co_fall:.2f} ns')

In [ ]:
print(f'Compatibilita con BGO NA e CO: {TestCompatibilita(tau_bgo_na_fall, err_bgo_na_fall, tau_bgo_co_fall, err_bgo_co_fall):.3g}')
print(f'Compatibilita con CSI NA e CO: {TestCompatibilita(tau_csi_na_fall, err_csi_na_fall, tau_csi_co_fall, err_csi_co_fall):.3g}')

__Confronto tempi di salita__

In [ ]:
# Salita

print(f'Rise BGO NA: {tau_bgo_na_rise:.2f} ± {err_bgo_na_rise:.2f} ns')
print(f'Rise BGO CO: {tau_bgo_co_rise:.2f} ± {err_bgo_co_rise:.2f} ns')
print('')
print(f'Rise CSI NA: {tau_csi_na_rise:.2f} ± {err_csi_na_rise:.2f} ns')
print(f'Rise CSI CO: {tau_csi_co_rise:.2f} ± {err_csi_co_rise:.2f} ns')

In [ ]:
print(f'Compatibilita con BGO NA e CO: {TestCompatibilita(tau_bgo_na_rise, err_bgo_na_rise, tau_bgo_co_rise, err_bgo_co_rise):.3g}')
print(f'Compatibilita con CSI NA e CO: {TestCompatibilita(tau_csi_na_rise, err_csi_na_rise, tau_csi_co_rise, err_csi_co_rise):.3g}')

## Lyso

In [ ]:
lyso = read_data('LySO/dat.CSV')
lyso = _crop_df(lyso, 10, .002)
lyso_rise, lyso_fall = rise_fall(lyso)

lyso_ers_rise = [.002] * len(lyso_rise['Time'])
lyso_ers_fall = [.002] * len(lyso_fall['Time'])

plt.plot(lyso_rise['Time'], lyso_rise['V'], label='Rise')
plt.plot(lyso_fall['Time'], lyso_fall['V'], label='Fall')
plt.legend()
plt.show()

In [ ]:
pars_lyso_rise, vals_lyso_rise, ers_lyso_rise, *other_lyso_rise = LS_fit(lyso_rise['Time'], lyso_rise['V'], lyso_ers_rise, exp_cre,
                                            a=.0035, b=21e6, c=0)

pars_lyso_fall, vals_lyso_fall, ers_lyso_fall, *other_lyso_fall = LS_fit(lyso_fall['Time'], lyso_fall['V'], lyso_ers_fall, exp_dec,
                                            a=1, b=0.01, c=0)

In [ ]:
tau_lyso_rise = 1 / vals_lyso_rise[1] * 1e9
err_lyso_rise = 1 / vals_lyso_rise[1]**2 * ers_lyso_rise[1] * 1e9

tau_lyso_fall = 1 / vals_lyso_fall[1] * 1e9
err_lyso_fall = 1 / vals_lyso_fall[1]**2 * ers_lyso_fall[1] * 1e9

print(f'Rise LySO: {tau_lyso_rise:.2f} ± {err_lyso_rise:.2f} ns')
print(f'Fall LySO: {tau_lyso_fall:.2f} ± {err_lyso_fall:.2f} ns')

# Parte finale per relazione

__Print finale per facilitare letture__

Usando solo il sodio perche' ci si fida di piu'

In [ ]:
print(f'BGO Rise and Fall: {tau_bgo_na_rise:.2f} ± {err_bgo_na_rise:.2f} ns, {tau_bgo_na_fall:.2f} ± {err_bgo_na_fall:.2f} ns')
print(f'Rapporto Rise/Fall BGO: {tau_bgo_na_rise / tau_bgo_na_fall:.2f}')
print('--------')
print(f'CSI Rise and Fall: {tau_csi_na_rise:.2f} ± {err_csi_na_rise:.2f} ns, {tau_csi_na_fall:.2f} ± {err_csi_na_fall:.2f} ns')
print(f'Rapporto Rise/Fall CSI: {tau_csi_na_rise / tau_csi_na_fall:.2f}')
print('--------')
print(f'Lyso Rise and Fall: {tau_lyso_rise:.2f} ± {err_lyso_rise:.2f} ns, {tau_lyso_fall:.2f} ± {err_lyso_fall:.2f} ns')
print(f'Rapporto Rise/Fall LySO: {tau_lyso_rise / tau_lyso_fall:.2f}')

In [ ]:
# plot per relazione

plt.figure(figsize=(8, 6))
plt.plot(bgo_co_rise['Time'], bgo_co_rise['V'], label='Rise', color='blue')
plt.plot(bgo_co_fall['Time'], bgo_co_fall['V'], label='Fall', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Tensione (V)')
plt.grid(linestyle='--', linewidth=0.5, alpha=0.7)
plt.legend()
plt.show()

__Commento sugli errori__

L'incertezza è stata presa all'oscilloscopio con la persistenza attiva e vedendo la larghezza della banda